In [21]:
import cvxpy as cp
import numpy as np
import scipy.linalg as la

Synthesis method 5.3 in https://arxiv.org/abs/1903.08599

In [22]:
nx = 10 #dimension of the state space  
nu = 4  #dimension of the control input space
nw = 2  #dimension of the disturbance input space
ny = 5  #dimension of the observation output space
nz = 3  #dimension of the performance output space

In [23]:
#The following define the state space representation of the plant
A = np.random.randn(nx,nx) #state to state
B1 = np.random.randn(nx,nw) #disturbance to state
B2 = np.random.randn(nx,nu) #control to state
C1 = np.random.randn(nz,nx) #state to performance
D11 = np.random.randn(nz,nw) #disturbance to performance
D12 = np.random.randn(nz,nu) #control to performance
C2 = np.random.randn(ny,nx) #state to observation
D21 = np.random.randn(ny,nw) #disturbance to observation
D22 = np.random.randn(ny,nu) #control to observation

In [24]:
#We will optimize over the following intermediate matrices 
#which will be used to produce the state space representation of the controller
An = cp.Variable((nx,nx))
Bn = cp.Variable((nx,ny))
Cn = cp.Variable((nu,nx))
Dn = cp.Variable((nu,ny))

X1 = cp.Variable((nx,nx),symmetric=True)
Y1 = cp.Variable((nx,nx),symmetric=True)
Z = cp.Variable((nz,nz),symmetric=True)

v = cp.Variable()

In [25]:
#Constraints
constraints = [v >= 0]

constraints += [cp.bmat([[A@Y1 + Y1@A.T + B2@Cn + Cn.T@B2.T, A + An.T + B2@Dn@C2,               B1 + B2@Dn@D21    ],
                         [(A + An.T + B2@Dn@C2).T,           X1@A + A.T@X1 + Bn@C2 + C2.T@Bn.T, X1@B1+Bn@D21      ],
                         [(B1 + B2@Dn@D21).T,                (X1@B1+Bn@D21).T,                  -1*np.identity(nw)]]) << 0]

constraints += [cp.bmat([[X1,                       np.identity(nx),            Y1@C1.T + Cn.T@D12.T  ],
                         [np.identity(nx),          Y1,                         C1.T + C2.T@Dn.T@D12.T],
                         [(Y1@C1.T + Cn.T@D12.T).T, (C1.T + C2.T@Dn.T@D12.T).T, Z                     ]]) >> 0]

constraints += [D11 + D12@Dn@D21 == 0]

constraints += [cp.bmat([[X1,              np.identity(nx)],
                         [np.identity(nx), Y1             ]]) >> 0]

constraints += [cp.trace(Z) <= v]

In [26]:
#Now put it all into a cvxpy problem and solve
prob = cp.Problem(cp.Minimize(v), constraints)
prob.solve(verbose=True)

                                     CVXPY                                     
                                     v1.4.1                                    
(CVXPY) Aug 20 05:25:40 PM: Your problem has 420 variables, 6 constraints, and 0 parameters.
(CVXPY) Aug 20 05:25:40 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 20 05:25:40 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 20 05:25:40 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Aug 20 05:25:40 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Aug 20 05:25:41 PM: Compiling problem (target solver=SCS).
(CVX

c:\Users\Jeff\mambaforge\envs\cvxpy\Lib\site-packages\cvxpy\problems\problem.py:1403: UserWarning: Solution may be inaccurate. Try another solver, adjusting the solver settings, or solve with verbose=True for more information.
  warnings.warn(


0.05171734155020571

In [27]:
#Now we can construct the controller
(X2, Y2t) = la.lu(np.identity(nx) - X1.value@Y1.value, permute_l = True)
Y2 = Y2t.T

In [35]:
F = np.block([[X2,X1.value@B2],[np.zeros((nu,nx)),np.identity(nu)]])
G = np.block([[An.value - X1.value@A@Y1.value,Bn.value],[Cn.value,Dn.value]])
H = np.block([[Y2.T, np.zeros((nx,ny))],[C2@Y1.value, np.identity(ny)]])
J = la.inv(F)@G@la.inv(H)
AK = J[:nx,:nx]
BK = J[:nx,nx:]
CK = J[nx:,:nx]
DK = J[nx:,nx:]

In [36]:
Dc = la.inv(np.identity(nu) + DK@D22)@DK
Cc = (np.identity(nu)+DK@D22)@CK
Bc = BK@(np.identity(ny)-D22@Dc)
Ac = AK - Bc@la.inv(np.identity(ny)-D22@Dc)@D22@Cc